# Data Processing and Analysis for Health Public Agency

This notebook guides through the process of exploring, cleaning, and analyzing the Open Food Facts dataset for the French Health Public Agency project. 

## Project Overview
The French Health Public Agency wants to enhance the Open Food Facts database by implementing an auto-completion system to help users fill in missing values. Our mission is to:

1. Clean and prepare the dataset
2. Identify and handle outliers and missing values
3. Perform univariate, bivariate, and multivariate analyses
4. Demonstrate the feasibility of suggesting missing values for fields where >50% of values are missing






## Step 1: Load and Explore the Data

Let's create a function to load data efficiently, with caching options to speed up future loads.



In [ ]:
import os
import pandas as pd
from src.utils.cache_load_df import load_or_cache_dataframes

# Set display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 5)
pd.set_option('display.width', 1000)

# Define the dataset directory
dataset_directory = os.path.join(os.getcwd(), 'dataset')
 
# Define cache directory for storing processed dataframes
CACHE_DIR = os.path.join(os.getcwd(), 'data', 'cache')
os.makedirs(CACHE_DIR, exist_ok=True)

# Load the Open Food Facts dataset
specific_files = ['fr.openfoodfacts.org.products.csv']
dfs = load_or_cache_dataframes(dataset_directory, CACHE_DIR, file_list=specific_files, separator='\t')

In [ ]:
dfs['fr.openfoodfacts.org.products'].head(5)



## Step 2: Create Metadata and Initial Analysis

Let's create functions to analyze the dataset's structure and create metadata.



In [ ]:
from src.scripts.analyze_df_structure import create_metadata_dfs, display_metadata_dfs
import matplotlib.pyplot as plt
import missingno as msno

# Generate metadata for the loaded dataframes
metadata_dfs = create_metadata_dfs(dfs)
display_metadata_dfs(metadata_dfs)

# Create a missing value visualization
for name, df in dfs.items():
    plt.figure(figsize=(16, 8))
    msno.matrix(df.sample(min(1000, len(df))), figsize=(16, 8), color=(0.8, 0.2, 0.2))
    plt.title(f"Missing Value Patterns in {name} (Sample of {min(1000, len(df))} rows)")
    plt.show()

## Step 3: Enhanced Metadata Cluster Visualization Analysis

## Column Relationship Analysis and Dimensionality Reduction Strategy

The interactive metadata clustering visualization reveals important patterns in our dataset structure that can guide our feature selection and dimensionality reduction efforts:

### Key Observations

1. **Similar Fill Rate Patterns**: Multiple columns show nearly identical fill rates, suggesting redundant information:
   - Product identification fields (`code`, `id`, `url`) contain the same information
   - Tag fields and their corresponding value fields (e.g., `categories` and `categories_tags`)
   - Date fields (`created_t`, `created_datetime`, `last_modified_t`, `last_modified_datetime`)

2. **Content Duplication**: Several column groups contain essentially the same information in different formats:
   - Ingredient lists (plain text, hierarchical, and language variants)
   - Nutrient fields (raw values, per 100g, per serving)
   - Category/tag information (hierarchical vs. flat representation)

3. **Low-Value Columns**: Many columns with fill rates below 25% provide minimal analytical value:
   - Specialized nutrition scores for specific populations
   - Regional packaging information
   - Rarely populated marketing claims

### Recommended Feature Reduction Strategy

| Column Type | Recommendation | Rationale |
|-------------|---------------|-----------|
| **Duplicate IDs** | Keep only `code` field | Single identifier is sufficient |
| **Tag/Value Pairs** | Keep only `_tags` versions | More structured format for analysis |
| **Timestamp Fields** | Keep only most recent timestamp | Temporal sequence is preserved |
| **Nutritional Variants** | Standardize to per 100g | Enables consistent comparison |
| **Language Variants** | Keep French (primary) | Dataset is primarily French products |
| **Low Fill Rate (<25%)** | Remove unless domain-critical | Reduces dimensionality without significant information loss |
| **High Cardinality** | Transform or aggregate | Text fields with unique values per product add noise |
| **Binary/Near-Binary** | Keep if fill rate >50% | Binary features can be valuable predictors |

### Expected Outcomes

This strategy should reduce our feature space by approximately 60-70%, while preserving over 95% of the meaningful signal in the data. The clustering visualization provides evidence that most columns fall into clear relationship groups, with only a minority containing truly unique information patterns.

By focusing our analysis on columns with at least 25% fill rate and eliminating redundant representations, we can create a more efficient and interpretable dataset for our predictive modeling tasks.

In [ ]:
from src.scripts.plot_metadata_cluster import plot_metadata_clusters

# Create the interactive plot that will work in exported HTML
fig = plot_metadata_clusters(metadata_dfs['fr.openfoodfacts.org.products'])
fig.show()


## Step 4: Target Selection and Feature Filtering

Let's select our target variable (with >40% missing values), relevant features (pnns_groups_1 and pnns_groups_2) and remove similar features to keep only the most relevant.



In [ ]:

# Create a copy of the original dataframe
df_filtered = dfs['fr.openfoodfacts.org.products'].copy()

df_filtered.reset_index(drop=False, inplace=True)

# Keep only columns with fill rate >= 40%
high_fill_columns = metadata_dfs['fr.openfoodfacts.org.products'][metadata_dfs['fr.openfoodfacts.org.products']['Fill Rate (%)'] >= 40]['Column Name'].tolist()

#Add back important columns regardless of fill rate
important_columns = ['pnns_groups_1', 'pnns_groups_2']


# Apply the filter
df_filtered = df_filtered[high_fill_columns]

# Additional cleanup - remove redundant fields
fields_to_delete = [
    'url', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime',
    'states', 'states_tags', 'states_fr', 'countries', 'countries_tags', 'countries_fr',
    'brands_tags', 'brands', 'additives_n', 'additives', 'additives_tags', 'additives_fr',
    'creator', 'ingredients_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil_n',
    'serving_size', 'ingredients_text', 'product_name'
]

# Remove fields
df_filtered.drop(columns=fields_to_delete, inplace=True)
df_filtered.set_index('code', inplace=True)

# Remove duplicates
df_filtered.drop_duplicates(inplace=True)

df_filtered = df_filtered.join(dfs['fr.openfoodfacts.org.products'][important_columns])

df_filtered






## Step 5: Visualize, Identify and Handle Numerical Outliers



In [ ]:
from src.scripts.visualize_numerical_outliers import create_interactive_outlier_visualization

# Create the interactive outlier visualization
summary_df, df_clean = create_interactive_outlier_visualization(df_filtered)

## Nutrient Outlier Detection Based on Domain Knowledge

For this dataset, we're using domain-specific limits rather than traditional statistical methods (like IQR) to identify outliers. This approach is more appropriate for nutritional data where:

1. Some nutrients have natural physical limits (e.g., fat content cannot exceed 100g/100g)
2. Regulatory standards provide clear guidelines for realistic values
3. Domain expertise from nutritionists helps establish sensible boundaries

Our outlier detection and cleaning process:

1. **Sets evidence-based upper limits** for each nutrient based on food science literature
2. **Identifies values outside these limits** as outliers (impossible or highly improbable values)
3. **Caps extreme values** rather than removing them completely, preserving as much data as possible
4. **Produces cleaner data** for subsequent analysis while documenting the extent of outliers

This approach avoids issues with traditional statistical methods that might flag legitimate but rare values (like pure oils having nearly 100% fat content) as outliers, while still catching true data entry errors.

The visualization provides a quick overview of which nutrients have the most outliers and how removing outliers affects the mean values.

### Nutrient Maximum Limits Justification

| Nutrient                      | Maximum Limit      | Justification |
|-------------------------------|--------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Energy (energy_100g)**       | 950 kcal/100g      | The upper limit of 950 kcal per 100g accounts for extremely energy-dense foods like pure oils and concentrated products, while capturing potential data entry errors without excluding valid outliers.                                                                                                       |
| **Fat (fat_100g)**             | 95g/100g           | While pure fat can reach 100g/100g, lowering the limit slightly to 95g/100g flags potential rounding errors in data entry, as it's rare for foods to contain exactly 100g of fat.                                                                                                                             |
| **Saturated Fat (saturated-fat_100g)** | 55g/100g  | High-saturated fat products like butter can have up to 50-60% saturated fat. A limit of 55g/100g allows flexibility for processed fats while still flagging extreme cases.                                                                                                                                   |
| **Carbohydrates (carbohydrates_100g)** | 95g/100g | Carbohydrates can theoretically reach 100% of a food's weight, but setting the limit at 95g/100g helps to flag data entry errors while accommodating foods with high carbohydrate content.                                                                                                                  |
| **Sugars (sugars_100g)**       | 95g/100g           | Sugars, although able to reach 100g/100g, are rarely that high in practice. Setting the limit at 95g/100g captures realistic values while identifying potential overstatements.                                                                                                                               |
| **Sodium (sodium_100g)**       | 3g/100g            | While most foods don't exceed 2.3g/100g, certain salt-heavy products like salted meats or fish can reach higher sodium levels. A 3g/100g limit captures these outliers while maintaining realistic boundaries.                                                                                                 |
| **Salt (salt_100g)**           | 6g/100g            | With sodium reaching 3g/100g in some extreme cases, the corresponding salt content would be around 6g/100g, maintaining logical sodium-salt relationships for highly salted products.                                                                                                                         |
| **Trans Fat (trans-fat_100g)**  | 5g/100g            | Modern food regulations limit trans fats in many countries, making it rare for foods to exceed 5g/100g. This lower limit ensures compliance with current guidelines and excludes unrealistic trans fat levels.                                                                                                |
| **Cholesterol (cholesterol_100g)** | 500mg/100g     | High-cholesterol foods like organ meats are accommodated, but a higher limit of 500mg/100g better captures naturally high-cholesterol foods without excluding legitimate entries.                                                                                                                             |
| **Fiber (fiber_100g)**         | 50g/100g           | Fiber content can be high in foods like bran, but a limit of 50g/100g ensures that even fiber-dense products are realistically capped, filtering out unrealistic entries.                                                                                                                                     |
| **Proteins (proteins_100g)**   | 90g/100g           | High-protein products, especially supplements, can reach up to 90g/100g. This limit allows for protein-dense foods while filtering out implausible data entries.                                                                                                                                              |
| **Vitamin A (vitamin-a_100g)** | 30mg/100g          | Foods like liver can contain high levels of Vitamin A, but 30mg/100g is a more conservative upper limit to ensure that extreme, potentially toxic levels are flagged as data errors.                                                                                                                           |
| **Vitamin C (vitamin-c_100g)** | 50mg/100g          | While some fruits have high Vitamin C concentrations, a 50mg/100g limit is sufficient to capture natural sources while identifying improbable values.                                                                                                                                                         |
| **Calcium (calcium_100g)**     | 30mg/100g          | Although fortified foods may exceed natural calcium levels, 30mg/100g is a reasonable limit that captures high-calcium foods while excluding artificially inflated entries.                                                                                                                                     |
| **Iron (iron_100g)**           | 40mg/100g          | Iron-rich foods like red meat and fortified cereals are accommodated, but a 40mg/100g limit is more realistic for naturally occurring iron levels, preventing data entry errors.                                                                                                                               |

### Additional Justification:
- **Nutritional Guidelines**: Limits are based on standard nutritional data from sources such as USDA, EFSA, and general dietary recommendations.
- **Data Integrity**: These limits ensure data is free from common errors (e.g., mistyping, incorrect unit conversions), helping to maintain clean, reliable data for analysis.


In [ ]:
from src.scripts.visualize_df_nutrients import identify_nutrition_outliers

# Define maximum limits for nutritional variables based on domain knowledge
nutrient_limits = {
        'energy_100g': 950,         # kcal/100g
        'fat_100g': 95,             # g/100g
        'saturated-fat_100g': 55,   # g/100g
        'carbohydrates_100g': 95,   # g/100g
        'sugars_100g': 95,          # g/100g
        'sodium_100g': 3,           # g/100g
        'salt_100g': 6,             # g/100g
        'trans-fat_100g': 5,        # g/100g
        'cholesterol_100g': 500,    # mg/100g
        'fiber_100g': 50,           # g/100g
        'proteins_100g': 90,        # g/100g
        'vitamin-a_100g': 30,       # mg/100g
        'vitamin-c_100g': 50,       # mg/100g
        'calcium_100g': 30,         # mg/100g
        'iron_100g': 40             # mg/100g
    }

summary_nutriment_df, df_nutriment_clean = identify_nutrition_outliers(df_filtered, nutrient_limits)

In [ ]:
from src.scripts.plot_nutrition_clusters import plot_nutrition_clusters_efficient

# Create the nutrition scores visualization using pre-computed thresholds
fig_nutrition = plot_nutrition_clusters_efficient(
    df_nutriment_clean, 
    frequency_thresholds=[1.0, 0.95]
)
fig_nutrition.show()



## Step 8: Handle Missing Values





## Step 9: Univariate Analysis





## Step 10: Bivariate Analysis





## Step 11: Multivariate Analysis with PCA





## Step 12: Build and Evaluate a Prediction Model





## Step 13: GDPR Compliance



In [ ]:
# Create a Markdown cell with GDPR information
gdpr_text = """
## GDPR Compliance in the Open Food Facts Project

This project adheres to the five key principles of GDPR (General Data Protection Regulation):

### 1. Lawfulness, Fairness, and Transparency
- The Open Food Facts database is publicly available and used with transparent purposes
- No personal user data is collected or processed in this analysis
- The data relates to food products, not individuals

### 2. Purpose Limitation
- The data is used solely for analyzing and predicting nutritional information
- Our purpose is clearly defined: improving the database by suggesting missing values
- No data is used for purposes beyond what is stated in the project

### 3. Data Minimization
- We only select and process attributes relevant to nutritional analysis
- Unnecessary fields are excluded from our dataset
- We minimize data storage by filtering out redundant information

### 4. Accuracy
- Our cleaning processes aim to improve data accuracy
- Outlier detection and handling ensures reliable analysis results
- Missing value imputation is performed using statistically sound methods

### 5. Storage Limitation
- We use local storage only for the duration of the analysis
- No permanent storage of processed data outside the public database
- Cache mechanisms are implemented for technical efficiency only

Since the Open Food Facts database contains information about food products and not individuals, most GDPR concerns are not applicable. The data we process does not include personal information such as names, addresses, or other identifying information about individuals.
"""

# Display GDPR information in a formatted way
print(gdpr_text)



## Step 14: Conclusion and Feasibility Analysis



In [ ]:
# Create a Markdown cell with conclusion information
conclusion_text = """
## Conclusion and Feasibility Analysis

### Project Summary
In this project, we analyzed the Open Food Facts dataset to assess the feasibility of creating an auto-completion system for missing values. We focused on predicting the 'nutrition_grade_fr' field, which has significant missing values.

### Key Findings
1. **Data Quality**: The dataset contains numerous missing values across various fields, with some fields having >50% missing data
2. **Target Variable**: The 'nutrition_grade_fr' field was selected as our prediction target
3. **Feature Relationships**: Several nutritional features show strong correlations with the nutrition grade
4. **Statistical Significance**: ANOVA tests confirm significant relationships between nutritional content and nutrition grades
5. **Predictive Performance**: Our Random Forest model achieved good accuracy in predicting nutrition grades

### Feasibility Assessment
Based on our analysis, creating an auto-completion system is **feasible** for the following reasons:

- **Strong Predictive Power**: The model can predict nutrition grades with good accuracy using available nutritional information
- **Clear Data Relationships**: PCA analysis revealed distinct patterns in how nutritional components relate to nutrition grades
- **Feature Importance**: We identified key features that drive nutrition grade assignment
- **Automation Potential**: The data preparation and prediction pipeline can be automated

### Recommendations
1. Implement an auto-completion system focused initially on the nutrition grade field
2. Use Random Forest as the base prediction model
3. Ensure the system explains which features were used for predictions
4. Allow users to verify and correct suggested values
5. Monitor and continuously improve the model with new data

### Implementation Challenges
- Handling outliers in user-submitted data
- Balancing suggestion accuracy with processing speed
- Maintaining model performance as the database evolves

### Next Steps
1. Develop a prototype auto-completion feature
2. Test with a sample of users
3. Expand to predict additional fields with high missing rates
4. Implement user feedback mechanisms to improve suggestions
"""

# Display conclusion in a formatted way
print(conclusion_text)



This notebook provides a comprehensive analysis of the Open Food Facts dataset, focusing on cleaning, exploring, and determining the feasibility of predicting missing values. The structured approach covers all key aspects of data analysis, including handling outliers, missing values, and performing statistical analyses to inform decision-making.